In [350]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import t

In [349]:
df = pd.read_csv('pmsi_analysis_v13.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,6,21,22,26,33,34,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [351]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [352]:
df['grp_cln'] = df['grp_cln'].astype('str')

In [353]:
df = df[df['grp_cln'] != 'no match']

In [354]:
df.head()

,finess,mois,annee,sexe,ghm2,GHS,age,duree,supp_rea,supp_si,...,top1_lib_dp_patients_SUV,top3_region_label_patients_SUV,top1_lib_dp_patients_PUMA,top3_region_label_patients_PUMA,top1_lib_dp_patients_AME,top3_region_label_patients_AME,top3_Libellé GHM_patients,region,effectif_region_2020,grp_cln
0,10007987,6,2012,1,20Z042,7268,36,9,0,0,...,"Syndrome de sevrage de l'alcool avec délirium,...",Hauts-de-France Île-de-France Guyane,Sevrage d'alcool,Hauts-de-France Auvergne-Rhône-Alpes Île-de-Fr...,Sevrage d'alcool,Île-de-France Auvergne-Rhône-Alpes Occitanie,"{'Ethylisme aigu, niveau 1': '12165', 'Ethylis...",GRAND EST,6009918.0,12
1,10007987,4,2013,1,20Z042,7268,38,6,0,0,...,"Syndrome de sevrage de l'alcool avec délirium,...",Hauts-de-France Île-de-France Guyane,Sevrage d'alcool,Hauts-de-France Auvergne-Rhône-Alpes Île-de-Fr...,Sevrage d'alcool,Île-de-France Auvergne-Rhône-Alpes Occitanie,"{'Ethylisme aigu, niveau 1': '12165', 'Ethylis...",GRAND EST,6009918.0,12
2,10007987,10,2015,1,20Z042,7268,39,8,0,0,...,"Syndrome de sevrage de l'alcool avec délirium,...",Hauts-de-France Île-de-France Guyane,Sevrage d'alcool,Hauts-de-France Auvergne-Rhône-Alpes Île-de-Fr...,Sevrage d'alcool,Île-de-France Auvergne-Rhône-Alpes Occitanie,"{'Ethylisme aigu, niveau 1': '12165', 'Ethylis...",GRAND EST,6009918.0,12
3,100000017,2,2015,1,20Z042,7282,51,20,0,0,...,"Syndrome de sevrage de l'alcool avec délirium,...",Hauts-de-France Île-de-France Guyane,Sevrage d'alcool,Hauts-de-France Auvergne-Rhône-Alpes Île-de-Fr...,Sevrage d'alcool,Île-de-France Auvergne-Rhône-Alpes Occitanie,"{'Ethylisme aigu, niveau 1': '12165', 'Ethylis...",GRAND EST,6009918.0,12
4,100000017,10,2016,2,20Z042,7282,53,12,0,0,...,"Syndrome de sevrage de l'alcool avec délirium,...",Hauts-de-France Île-de-France Guyane,Sevrage d'alcool,Hauts-de-France Auvergne-Rhône-Alpes Île-de-Fr...,Sevrage d'alcool,Île-de-France Auvergne-Rhône-Alpes Occitanie,"{'Ethylisme aigu, niveau 1': '12165', 'Ethylis...",GRAND EST,6009918.0,12


### Surcôut précarité

In [355]:
df_oc = df.groupby(['grp_cln','raison']).agg({'cost':'mean'})
df_oc = df_oc.reset_index()

In [356]:
df_oc = df_oc[df_oc['grp_cln']!='no match']

In [357]:
df_oc = df_oc.pivot_table('cost', ['grp_cln'], 'raison')
df_oc = df_oc.reset_index()
df_oc = df_oc.rename_axis(None, axis=1)
df_oc['grp_cln'] = df_oc['grp_cln'].astype('int')
df_oc = df_oc.sort_values(by='grp_cln')
df_oc.index = range(1,len(df_oc)+1)

In [358]:
df_oc

,grp_cln,AME,PUMA,SUV
1,1,2575.531051,2486.693800,2685.197574
2,2,678.749469,680.822475,1020.010530
3,3,4193.720169,4000.914447,4533.917557
4,4,984.439284,987.652134,998.567391
5,5,967.709538,819.338891,1276.441130
6,6,3360.046379,3308.411456,3524.491989
7,7,989.769207,1028.906983,1015.790149
8,8,5466.127951,1534.693416,7362.529219
9,9,10616.495148,4099.124881,12288.016027
10,10,869.167391,825.832131,957.906373


In [359]:
def generate_ic(grp_cln,raison_1,raison_2,conf=0.95):
    
    r1= np.array(df[(df['grp_cln']==grp_cln) &  (df['raison']==raison_1)]['cost'])
    r2= np.array(df[(df['grp_cln']==grp_cln) &  (df['raison']==raison_2)]['cost'])
    size = min(len(r1),len(r2))
    oc = np.random.choice(r1,size=size) - np.random.choice(r2,size=size)
    
    return sms.DescrStatsW(oc).tconfint_mean()
    

#### SUV vs AME

In [360]:
suv_ame_ic_le = list()
suv_ame_ic_he = list()

for grp_cln in range(1,21):
    
    ic = generate_ic(str(grp_cln),'SUV','AME')
    suv_ame_ic_le.append(ic[0])
    suv_ame_ic_he.append(ic[1])

In [361]:
df_oc['SUV_vs_AME'] = df_oc['SUV'] - df_oc['AME']

In [362]:
df_oc['suv_ame_ic_le'] = suv_ame_ic_le
df_oc['suv_ame_ic_he'] = suv_ame_ic_he

In [363]:
df_oc

,grp_cln,AME,PUMA,SUV,SUV_vs_AME,suv_ame_ic_le,suv_ame_ic_he
1,1,2575.531051,2486.693800,2685.197574,109.666523,96.722039,159.096131
2,2,678.749469,680.822475,1020.010530,341.261061,268.487987,368.087763
3,3,4193.720169,4000.914447,4533.917557,340.197388,199.895617,362.432593
4,4,984.439284,987.652134,998.567391,14.128107,13.592696,16.522334
5,5,967.709538,819.338891,1276.441130,308.731592,253.026095,362.479310
6,6,3360.046379,3308.411456,3524.491989,164.445610,45.161615,242.482905
7,7,989.769207,1028.906983,1015.790149,26.020942,-1.134544,35.356436
8,8,5466.127951,1534.693416,7362.529219,1896.401268,621.151789,2458.274076
9,9,10616.495148,4099.124881,12288.016027,1671.520879,-2206.633455,1992.414136
10,10,869.167391,825.832131,957.906373,88.738982,-63.962456,176.590179


#### AME vs PUMA

In [364]:
ame_puma_ic_le = list()
ame_puma_ic_he = list()

for grp_cln in range(1,21):
    
    ic = generate_ic(str(grp_cln),'AME','PUMA')
    ame_puma_ic_le.append(ic[0])
    ame_puma_ic_he.append(ic[1])

In [365]:
df_oc['AME_vs_PUMA'] = df_oc['AME'] - df_oc['PUMA']

In [366]:
df_oc['ame_puma_ic_le'] = ame_puma_ic_le
df_oc['ame_puma_ic_he'] = ame_puma_ic_he

In [367]:
df_oc

,grp_cln,AME,PUMA,SUV,SUV_vs_AME,suv_ame_ic_le,suv_ame_ic_he,AME_vs_PUMA,ame_puma_ic_le,ame_puma_ic_he
1,1,2575.531051,2486.693800,2685.197574,109.666523,96.722039,159.096131,88.837251,71.078387,91.101392
2,2,678.749469,680.822475,1020.010530,341.261061,268.487987,368.087763,-2.073006,-6.668369,8.936514
3,3,4193.720169,4000.914447,4533.917557,340.197388,199.895617,362.432593,192.805722,140.647823,209.474315
4,4,984.439284,987.652134,998.567391,14.128107,13.592696,16.522334,-3.212849,-3.975622,-1.061505
5,5,967.709538,819.338891,1276.441130,308.731592,253.026095,362.479310,148.370647,105.255386,152.007592
6,6,3360.046379,3308.411456,3524.491989,164.445610,45.161615,242.482905,51.634922,-32.135324,107.205635
7,7,989.769207,1028.906983,1015.790149,26.020942,-1.134544,35.356436,-39.137776,-41.188630,-35.860669
8,8,5466.127951,1534.693416,7362.529219,1896.401268,621.151789,2458.274076,3931.434535,3644.367533,4108.406027
9,9,10616.495148,4099.124881,12288.016027,1671.520879,-2206.633455,1992.414136,6517.370267,6453.187623,7378.364471
10,10,869.167391,825.832131,957.906373,88.738982,-63.962456,176.590179,43.335260,24.622728,58.620938


#### SUV vs PUMA

In [368]:
suv_puma_ic_le = list()
suv_puma_ic_he = list()

for grp_cln in range(1,21):
    
    ic = generate_ic(str(grp_cln),'SUV','PUMA')
    suv_puma_ic_le.append(ic[0])
    suv_puma_ic_he.append(ic[1])

In [369]:
df_oc['SUV_vs_PUMA'] = df_oc['SUV'] - df_oc['PUMA']

In [370]:
df_oc['suv_puma_ic_le'] = suv_puma_ic_le
df_oc['suv_puma_ic_he'] = suv_puma_ic_he

In [371]:
df_oc

,grp_cln,AME,PUMA,SUV,SUV_vs_AME,suv_ame_ic_le,suv_ame_ic_he,AME_vs_PUMA,ame_puma_ic_le,ame_puma_ic_he,SUV_vs_PUMA,suv_puma_ic_le,suv_puma_ic_he
1,1,2575.531051,2486.693800,2685.197574,109.666523,96.722039,159.096131,88.837251,71.078387,91.101392,198.503774,188.004461,247.897552
2,2,678.749469,680.822475,1020.010530,341.261061,268.487987,368.087763,-2.073006,-6.668369,8.936514,339.188055,289.126683,380.632816
3,3,4193.720169,4000.914447,4533.917557,340.197388,199.895617,362.432593,192.805722,140.647823,209.474315,533.003110,415.360996,576.179592
4,4,984.439284,987.652134,998.567391,14.128107,13.592696,16.522334,-3.212849,-3.975622,-1.061505,10.915258,8.848508,11.784319
5,5,967.709538,819.338891,1276.441130,308.731592,253.026095,362.479310,148.370647,105.255386,152.007592,457.102239,435.671825,534.135252
6,6,3360.046379,3308.411456,3524.491989,164.445610,45.161615,242.482905,51.634922,-32.135324,107.205635,216.080533,129.119445,299.527683
7,7,989.769207,1028.906983,1015.790149,26.020942,-1.134544,35.356436,-39.137776,-41.188630,-35.860669,-13.116835,-25.088164,10.927112
8,8,5466.127951,1534.693416,7362.529219,1896.401268,621.151789,2458.274076,3931.434535,3644.367533,4108.406027,5827.835802,4605.500849,6022.680185
9,9,10616.495148,4099.124881,12288.016027,1671.520879,-2206.633455,1992.414136,6517.370267,6453.187623,7378.364471,8188.891146,6634.312797,10208.629883
10,10,869.167391,825.832131,957.906373,88.738982,-63.962456,176.590179,43.335260,24.622728,58.620938,132.074242,48.249376,366.698220


In [372]:
df_oc['range_suv_ame'] = df_oc['suv_ame_ic_he']-df_oc['suv_ame_ic_le']
df_oc['range_ame_puma'] = df_oc['ame_puma_ic_he']-df_oc['ame_puma_ic_le']
df_oc['range_suv_puma'] = df_oc['suv_puma_ic_he']-df_oc['suv_puma_ic_le']


In [373]:
df_oc

,grp_cln,AME,PUMA,SUV,SUV_vs_AME,suv_ame_ic_le,suv_ame_ic_he,AME_vs_PUMA,ame_puma_ic_le,ame_puma_ic_he,SUV_vs_PUMA,suv_puma_ic_le,suv_puma_ic_he,range_suv_ame,range_ame_puma,range_suv_puma
1,1,2575.531051,2486.693800,2685.197574,109.666523,96.722039,159.096131,88.837251,71.078387,91.101392,198.503774,188.004461,247.897552,62.374092,20.023005,59.893090
2,2,678.749469,680.822475,1020.010530,341.261061,268.487987,368.087763,-2.073006,-6.668369,8.936514,339.188055,289.126683,380.632816,99.599776,15.604883,91.506133
3,3,4193.720169,4000.914447,4533.917557,340.197388,199.895617,362.432593,192.805722,140.647823,209.474315,533.003110,415.360996,576.179592,162.536976,68.826491,160.818595
4,4,984.439284,987.652134,998.567391,14.128107,13.592696,16.522334,-3.212849,-3.975622,-1.061505,10.915258,8.848508,11.784319,2.929638,2.914118,2.935811
5,5,967.709538,819.338891,1276.441130,308.731592,253.026095,362.479310,148.370647,105.255386,152.007592,457.102239,435.671825,534.135252,109.453214,46.752206,98.463427
6,6,3360.046379,3308.411456,3524.491989,164.445610,45.161615,242.482905,51.634922,-32.135324,107.205635,216.080533,129.119445,299.527683,197.321290,139.340960,170.408237
7,7,989.769207,1028.906983,1015.790149,26.020942,-1.134544,35.356436,-39.137776,-41.188630,-35.860669,-13.116835,-25.088164,10.927112,36.490980,5.327961,36.015277
8,8,5466.127951,1534.693416,7362.529219,1896.401268,621.151789,2458.274076,3931.434535,3644.367533,4108.406027,5827.835802,4605.500849,6022.680185,1837.122286,464.038494,1417.179336
9,9,10616.495148,4099.124881,12288.016027,1671.520879,-2206.633455,1992.414136,6517.370267,6453.187623,7378.364471,8188.891146,6634.312797,10208.629883,4199.047591,925.176847,3574.317086
10,10,869.167391,825.832131,957.906373,88.738982,-63.962456,176.590179,43.335260,24.622728,58.620938,132.074242,48.249376,366.698220,240.552635,33.998210,318.448844


In [375]:
df_oc.to_csv('over_cost_ic_fin.csv')